<a href="https://colab.research.google.com/github/rajashekar/colab/blob/main/wake_word_detection_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Architecture & flow diagram - https://firebasestorage.googleapis.com/v0/b/firescript-577a2.appspot.com/o/imgs%2Fapp%2Frajashekar%2FQPitMq4MPE.png?alt=media&token=9daae34a-0c80-4920-91bc-bd03eb6661be

<img src="https://firebasestorage.googleapis.com/v0/b/firescript-577a2.appspot.com/o/imgs%2Fapp%2Frajashekar%2FQPitMq4MPE.png?alt=media&token=9daae34a-0c80-4920-91bc-bd03eb6661be" width="2500" height="500"/>

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!cat /proc/meminfo | head -1

MemTotal:       13305368 kB


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/MyDrive/Colab/Fourth.Brain/wake_word_detection/howl

/content/gdrive/MyDrive/Colab/Fourth.Brain/wake_word_detection/howl


In [5]:
!python --version

Python 3.7.11


In [6]:
# install dependencies
# pocketsphinx requires https://github.com/bambocher/pocketsphinx-python#install-requirements
!sudo apt-get install -qq python python-dev python-pip build-essential swig git libpulse-dev libasound2-dev
# pyaudio
!sudo apt-get install python-pyaudio python3-pyaudio

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 26.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libpulse-mainloop-glib0:amd64.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../00-libpulse-mainloop-glib0_1%3a11.1-1ubuntu7.11_amd64.deb ...
Unpacking libpulse-mainloop-glib0:amd64 (1:11.1-1ubuntu7.11) ...
Selecting previously unselected package libpulse-dev:amd64.
Preparing to unpack .../01-libpulse-dev_1%3a11.1-1ubuntu7.11_amd64.deb ...
Unpacking libpulse-dev:amd64 (1:11.1-1ubuntu7.11) ...
Selecting previously unselected package libpython-all-dev:amd64.
P

In [ ]:
# unrar data
!unrar x -o- data.rar /content/

In [8]:
# load data train meta data
import json
from pathlib import Path
def read_meta_data(path, jsonl_name):
  path = Path(path)
  metadata_list = []
  with open(path/jsonl_name) as f:
      for json_str in iter(f.readline, ""):
          metadata = json.loads(json_str)
          # metadata['path'] = f"{path}/audio/{metadata['path']}"
          metadata['path'] = (path / "audio" /  metadata['path']).absolute()
          metadata_list.append(metadata)
  return metadata_list

In [9]:
postive_data_path = "/content/data/hey-fourth-brain-positive/"
train_pos_metadata = read_meta_data(postive_data_path, "aligned-metadata-training.jsonl")
dev_pos_metadata = read_meta_data(postive_data_path, "aligned-metadata-dev.jsonl")
test_pos_metadata = read_meta_data(postive_data_path, "aligned-metadata-test.jsonl")

In [10]:
train_pos_metadata[0]

{'end_timestamps': [480.0,
  730.0,
  730.0,
  730.0,
  835.0,
  940.0,
  940.0,
  940.0,
  1021.25,
  1102.5,
  1183.75,
  1265.0,
  1346.25,
  1427.5,
  1508.75,
  1590.0,
  1590.0,
  1590.0,
  1663.3333333333333,
  1736.6666666666667,
  1810.0,
  1810.0,
  1810.0,
  1900.0,
  1990.0,
  1990.0,
  1990.0,
  2075.0,
  2160.0,
  2245.0,
  2330.0,
  2330.0,
  2330.0,
  2580.0,
  2580.0,
  2580.0,
  2625.0,
  2670.0,
  2670.0,
  2670.0,
  2776.0,
  2882.0,
  2988.0,
  3094.0,
  3200.0,
  3200.0,
  3200.0,
  3350.0,
  3500.0,
  3650.0,
  3650.0,
  3770.0,
  3838.0,
  3906.0,
  3974.0,
  4042.0,
  4110.0,
  4110.0,
  4110.0,
  4235.0,
  4360.0,
  4485.0,
  4610.0,
  4610.0,
  4610.0,
  4890.0,
  4890.0,
  4890.0,
  5015.0,
  5140.0,
  5265.0,
  5390.0,
  5515.0,
  5640.0],
 'path': PosixPath('/content/data/hey-fourth-brain-positive/audio/common_voice_en_20433916.wav'),
 'phone_end_timestamps': None,
 'phone_strings': None,
 'transcription': 'he was decorated with the order of the golden kit

In [11]:
print(f"Train samples in positive dataset {len(train_pos_metadata)}")
print(f"Dev samples in positive dataset {len(dev_pos_metadata)}")
print(f"Test samples in positive dataset {len(test_pos_metadata)}")

Train samples in positive dataset 828
Dev samples in positive dataset 29
Test samples in positive dataset 19


In [12]:
negative_data_path = "/content/data/hey-fourth-brain-negative/"
train_neg_metadata = read_meta_data(negative_data_path, "aligned-metadata-training.jsonl")
dev_neg_metadata = read_meta_data(negative_data_path, "aligned-metadata-dev.jsonl")
test_neg_metadata = read_meta_data(negative_data_path, "aligned-metadata-test.jsonl")

In [13]:
print(f"Train samples in negative dataset {len(train_neg_metadata)}")
print(f"Dev samples in negative dataset {len(dev_neg_metadata)}")
print(f"Test samples in negative dataset {len(test_neg_metadata)}")

Train samples in negative dataset 11254
Dev samples in negative dataset 322
Test samples in negative dataset 312


In [14]:
stiched_data_path = "/content/data/hey-fourth-brain-stiched/"
train_stiched_metadata = read_meta_data(stiched_data_path, "aligned-metadata-training.jsonl")
dev_stiched_metadata = read_meta_data(stiched_data_path, "aligned-metadata-dev.jsonl")
test_stiched_metadata = read_meta_data(stiched_data_path, "aligned-metadata-test.jsonl")

In [15]:
print(f"Train samples in stiched dataset {len(train_stiched_metadata)}")
print(f"Dev samples in stiched dataset {len(dev_stiched_metadata)}")
print(f"Test samples in stiched dataset {len(test_stiched_metadata)}")

Train samples in stiched dataset 5000
Dev samples in stiched dataset 2500
Test samples in stiched dataset 2500


In [16]:
train_ds = train_pos_metadata + train_neg_metadata + train_stiched_metadata
dev_ds = dev_pos_metadata + dev_neg_metadata + dev_stiched_metadata
test_ds = test_pos_metadata + test_neg_metadata + test_stiched_metadata

In [17]:
print(f"Total Train samples {len(train_ds)}")
print(f"Total Dev samples {len(dev_ds)}")
print(f"Total test samples {len(test_ds)}")

Total Train samples 17082
Total Dev samples 2851
Total test samples 2831


In [18]:
vocab = ["hey", "fourth", "brain"]
inference_sequence = [0, 1, 2]
vocab_map = dict(zip(vocab, inference_sequence))

In [19]:
inference_sequence_str = ''.join(map(str, inference_sequence))
def search_vocab(transcription):
  exists = False
  final = ""
  for word in transcription.lower().split():
    if word in vocab:
      final += str(vocab_map[word])
    else:
      final += str(len(inference_sequence))
  
  if final == inference_sequence_str:
    exists = True
  return exists

In [20]:
# dev containing vocab
dev_pos_ds = list(filter(lambda x: search_vocab(x['transcription']), dev_ds))
# dev NOT containing vocab
dev_neg_ds = list(filter(lambda x: not search_vocab(x['transcription']), dev_ds))
# test containing vocab
test_pos_ds = list(filter(lambda x: search_vocab(x['transcription']), test_ds))
# test NOT containing vocab
test_neg_ds = list(filter(lambda x: not search_vocab(x['transcription']), test_ds))

In [21]:
test_pos_ds[0]

{'end_timestamps': [0.0,
  35.0,
  70.0,
  70.0,
  70.0,
  140.0,
  210.0,
  280.0,
  349.9999999999998,
  419.9999999999998,
  419.9999999999998,
  419.9999999999998,
  524.9999999999998,
  629.9999999999998,
  734.9999999999998,
  839.9999999999998],
 'path': PosixPath('/content/data/hey-fourth-brain-stiched/audio/6010.wav'),
 'phone_end_timestamps': None,
 'phone_strings': None,
 'transcription': 'hey fourth brain',
 'word_end_timestamps': None,
 'words': None}

In [22]:
import librosa
import warnings

def load_audio_data(path: str, sr: int = 16000, mono: bool = True):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        print(f"loading {path}")
        return librosa.core.load(path, sr=sr, mono=mono)[0]

In [23]:
def compute_labels(metadata):
        frame_labels = dict()
        start_timestamp = []
        char_indices = []

        char_idx = 0
        for word in metadata["transcription"].lower().split():
          if word in vocab:
            vocab_found = True
            word_size = len(word.rstrip())

            # if the current word is in vocab, store necessary informations
            if vocab_found:
                label = vocab_map[word]
                end_timestamp = metadata['end_timestamps'][char_idx + word_size - 1]
                frame_labels[end_timestamp] = label
                char_indices.append((label, list(range(char_idx, char_idx + word_size))))
                start_timestamp.append((label, metadata['end_timestamps'][char_idx - 1] if char_idx > 0 else 0.0))

            char_idx += word_size + 1  # space

        return  {
            "timestamp_label_map": frame_labels,
            "start_timestamp": start_timestamp,
            "char_indices": char_indices
        }

In [89]:
import random
import torch
import numpy as np

def process_batch(batch):
    print(type(batch))
    print(len(batch))
    sample_rate = 16000
    window_size_ms = 750
    negative_label = len(vocab)
    examples = []
    max_window_size = 16000
    max_length = int(window_size_ms / 1000 * sample_rate)

    for sample in batch:
      # print(sample)
      example = {}
      example['audio_data'] = torch.from_numpy(load_audio_data(sample['path']))
      example['label_data'] = compute_labels(sample)
      # if the sample does not contain any sample value
      if not example['label_data']["timestamp_label_map"]:
        if example['audio_data'].size(-1) < max_length:
          examples.append((negative_label, example['audio_data']))
        else:
          a = random.randint(0, example['audio_data'].size(-1) - max_length)
          examples.append((negative_label, example['audio_data'][..., a:a + max_window_size]))
        continue
      
      end_ms, label = random.choice(list( example['label_data']["timestamp_label_map"].items()))
      end_ms_rand = end_ms + (random.random() * 20)
      b = int((end_ms_rand / 1000) * sample_rate)
      a = max(b - int((window_size_ms / 1000) * sample_rate), 0)
      examples.append((label, example['audio_data'][..., a:b])) 
    
    labels_lst = [x[0] for x in examples]
    audio_ds = [x[1] for x in examples]
    lengths=[x[1].size(-1) for x in examples]

    print(labels_lst)
    print(lengths)

    lengths_nd = np.array(lengths)
    sort_indices = np.argsort(-lengths_nd)
    audio_data_lst = np.array(audio_ds, dtype=object)[sort_indices].tolist()
    labels_lst = np.array(labels_lst, dtype=object)[sort_indices].tolist()
    lengths = np.array(lengths, dtype=object)[sort_indices].tolist()
    
    audio_tensor = []
    for audio_data in audio_data_lst:
      if audio_data.size(-1) > max_length:
        audio_data = audio_data[:max_length]
      squeezed_data = audio_data.squeeze()
      if squeezed_data.dim() == 0:
          squeezed_data = squeezed_data.unsqueeze(0)
      if random.random() < 0.5:
          x = (torch.zeros(max_length - audio_data.size(-1)), squeezed_data)
      else:
          x = (squeezed_data, torch.zeros(max_length - audio_data.size(-1)))
      audio_tensor.append(torch.cat(x, -1)) 


    batch_tensor = {
      'audio_data' : torch.stack(audio_tensor),
      'labels' : torch.tensor(labels_lst),
      'lengths' : torch.tensor(lengths)
    }

    return batch_tensor
    

In [90]:
import torch.utils.data as tud

batch_size = 16
num_workers = 0

train_dl = tud.DataLoader(train_ds,
                  batch_size=batch_size,
                  drop_last=True,
                  shuffle=True,
                  num_workers=num_workers,
                  collate_fn=process_batch)

In [92]:
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CUDA is not available.  Inference on CPU ...')
else:
    print('CUDA is available!  Inference on GPU ...')

CUDA is not available.  Inference on CPU ...


In [97]:
import torch.nn as nn
import torch.nn.functional as F

class SequentialCnn():
    def __init__(self, num_labels, num_maps1, num_maps2, num_hidden_input, hidden_size):
        super().__init__()
        conv0 = nn.Conv2d(1, num_maps1, (20, 16), padding=(10, 0), stride=(1, 2), bias=True)
        pool = nn.MaxPool2d(2)
        conv1 = nn.Conv2d(num_maps1, num_maps2, (5, 5), padding=2, stride=(2, 1), bias=True)
        self.encoder1 = nn.Sequential(conv0,
                                      nn.ReLU(),
                                      pool,
                                      nn.BatchNorm2d(num_maps1, affine=True))
        self.encoder2 = nn.Sequential(conv1,
                                      nn.ReLU(),
                                      pool,
                                      nn.BatchNorm2d(num_maps2, affine=True))
        self.output = nn.Sequential(nn.Linear(3 * num_maps2, hidden_size),
                                    nn.ReLU(),
                                    nn.Dropout(0.1),
                                    nn.Linear(hidden_size, num_labels))

    def forward(self, x, lengths):
        x = x[:, :1]  # log-Mels only
        x = x.permute(0, 1, 3, 2)  # (time, frequency)
        x1 = self.encoder1(x)
        x2 = self.encoder2(x1)
        x = x2.permute(2, 0, 1, 3).contiguous()
        x = x.view(x.size(0), x.size(1), x.size(2) * x.size(3))
        return self.output(x)

In [98]:
num_labels = len(vocab)
num_maps1  = 48
num_maps2  = 64
num_hidden_input =  384
hidden_size = 128
model = SequentialCnn(num_labels, num_maps1, num_maps2, num_hidden_input, hidden_size)

In [99]:
if train_on_gpu:
  model.cuda()

In [91]:
epochs = 2
for epoch in range(epochs):
  for batch in train_dl:
    audio_data = batch['audio_data']
    labels = batch['labels']
    lengths = batch['lengths']

    if train_on_gpu:
      audio_data.cuda()
      labels.cuda()
      lengths.cuda()

    # TODO 
    # Transformations
    # Train
    # Evaluate  

    

Streaming output truncated to the last 5000 lines.
loading /content/data/hey-fourth-brain-positive/audio/common_voice_en_20018359.wav
loading /content/data/hey-fourth-brain-stiched/audio/2846.wav
loading /content/data/hey-fourth-brain-stiched/audio/4225.wav
loading /content/data/hey-fourth-brain-negative/audio/common_voice_en_20327812.wav
loading /content/data/hey-fourth-brain-stiched/audio/2757.wav
loading /content/data/hey-fourth-brain-negative/audio/common_voice_en_19099427.wav
loading /content/data/hey-fourth-brain-stiched/audio/3110.wav
loading /content/data/hey-fourth-brain-negative/audio/common_voice_en_23889446.wav
loading /content/data/hey-fourth-brain-stiched/audio/2659.wav
loading /content/data/hey-fourth-brain-negative/audio/common_voice_en_19618424.wav
[3, 0, 3, 3, 0, 2, 1, 0, 0, 3, 1, 3, 2, 3, 1, 3]
[16000, 4509, 16000, 16000, 2673, 12000, 12000, 4370, 6457, 16000, 11794, 16000, 11807, 16000, 12000, 16000]
{'audio_data': tensor([[ 3.9673e-03, -9.7656e-04,  5.7068e-03,  ..

KeyboardInterrupt: ignored